In [1]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import datasets
import numpy as np
import pandas as pd

In [2]:
TREE_PARAMS_DICT = {
    'max_depth': 2,
    'min_samples_split': 0.5,
    'criterion': 'squared_error'
}

TAU = 0.05

In [3]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters, tau):
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        self.estimators = []

    def mse_grad(self, y_true, y_pred):
        return (y_true - y_pred)
        
        
    def fit(self, X_data, y_data):
        self.base_algo = DecisionTreeRegressor(**self.tree_params_dict)
        self.base_algo.fit(X_data, y_data)
        curr_pred = self.base_algo.predict(X_data)
        for iter_num in range(self.iters):
            grad = self.mse_grad(y_data, curr_pred)
            algo = DecisionTreeRegressor(**self.tree_params_dict)
            algo.fit(X_data, grad)
            self.estimators.append(algo)
            curr_pred += self.tau * algo.predict(X_data)
        return self
    
    def predict(self, X_data):
        res = self.base_algo.predict(X_data)
        for estimator in self.estimators:
            res += self.tau * estimator.predict(X_data)
        return res

Подгрузим данные.

In [4]:
boston = datasets.load_boston()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [5]:
data = boston['data']
target = boston['target']

In [6]:
from sklearn.ensemble import GradientBoostingRegressor

In [7]:
gbr = GradientBoostingRegressor(max_depth=3, random_state=42)

Качество реализации GBR из sklearn.

In [8]:
sklearn_score = np.mean(cross_val_score(gbr, data, target, cv=3, scoring='neg_mean_squared_error'))
sklearn_score

-27.641784915305465

In [9]:
algo = SimpleGB(tree_params_dict=TREE_PARAMS_DICT, iters=700, tau=TAU)

In [10]:
my_score = np.mean(cross_val_score(algo, data, target, cv=3, scoring='neg_mean_squared_error'))
my_score

-28.871627384203265

In [11]:
(sklearn_score - my_score) / sklearn_score

-0.04449215101940929